In [1]:
!export WANDB_DISABLED=true

In [2]:
%load_ext autoreload
%autoreload 2
from utils import *
from trainer import Trainer
#import wandb

#wandb.login(key="")
device = 'cuda:0'
base_model = HookedTransformer.from_pretrained(
    "gemma-2-2b", 
    device=device, 
)
chat_model = HookedTransformer.from_pretrained(
    "gemma-2-2b-it", 
    device=device, 
)

2025-05-16 21:37:47.543704: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-16 21:37:47.683468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747431467.745821    5196 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747431467.763193    5196 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-16 21:37:47.900473: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b into HookedTransformer


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b-it into HookedTransformer


In [ ]:
pip install transformer_lens

In [ ]:
pip install einops


In [ ]:
pip install huggingface_hub

In [3]:
from huggingface_hub import login
login("")

In [4]:
all_tokens = load_pile_lmsys_mixed_tokens()
import os
os.environ["WANDB_DISABLED"] = "true"
import wandb
wandb.init(mode="disabled")
default_cfg = {
    "seed": 49,
    "batch_size": 8192,
    "buffer_mult": 128,
    "eta": 1.5, # controls relative weight of exclusive vs shared latents
    "lr": 5e-5,
    "num_tokens": 250_000_000,
    "l1_coeff": 2,
    "beta1": 0.9,
    "beta2": 0.999,
    "d_in": base_model.cfg.d_model,
    "dict_size": 2**14,
    "seq_len": 1024,
    "enc_dtype": "fp32",
    "model_name": "gemma-2-2b",
    "site": "resid_pre",
    "device": "cuda:0",
    "model_batch_size": 8,
    "log_every": 100,
    "save_every": 3000,
    "dec_init_norm": 0.08,
    "hook_point": "blocks.14.hook_resid_pre",
    "wandb_project": "",
    "wandb_entity": "",
}
cfg = arg_parse_update_cfg(default_cfg)

trainer = Trainer(cfg, base_model, chat_model, all_tokens)
trainer.train()

Loading data from disk


/home/user/tied-crosscoder-Matthew/utils.py:184: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



In IPython - skipped argparse


Estimating norm scaling factor: 100%|█████████| 100/100 [02:44<00:00,  1.65s/it]


Refreshing the buffer!


  0%|                                                   | 0/129 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 7.81 GiB. GPU 0 has a total capacity of 79.20 GiB of which 1.57 GiB is free. Including non-PyTorch memory, this process has 77.63 GiB memory in use. Of the allocated memory 71.67 GiB is allocated by PyTorch, and 5.30 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
dir(trainer)